In [1]:
import numpy as np
import tensorflow as tf

In [2]:
x = tf.placeholder("float", shape=[None,32,32,3])
y_ = tf.placeholder(tf.int32, shape=[None])
keep_prob = tf.placeholder("float")

In [3]:
def im_pre(image):
#    new_img = tf.image.random_brightness(image, max_delta=63) #随机调节图像的亮度  
#    new_img = tf.image.random_flip_left_right(new_img) #随机地左右翻转图像  
#    new_img = tf.image.random_contrast(new_img, lower=0.2, upper=1.8) #随机地调整图像对比度  
    new_img = tf.image.per_image_standardization(image)
    return new_img

In [4]:
img =  tf.map_fn(im_pre,x)

In [5]:
b_size = 128
learning_rate = 0.0001

In [6]:
reg = tf.contrib.layers.l2_regularizer(scale=0.1) 

In [7]:
conv2d_1 = tf.contrib.layers.convolution2d(
    img,
    num_outputs=32,
    weights_initializer=tf.truncated_normal_initializer(stddev=0.0001),
    weights_regularizer = reg,
    kernel_size = (5,5),
    activation_fn = tf.nn.relu,
    stride = (1,1),
    padding='SAME',
    trainable = True)
pool_1 = tf.nn.max_pool(conv2d_1,
                       ksize = [1,3,3,1],
                       strides = [1,2,2,1],
                       padding='SAME')

In [8]:
conv2d_2 = tf.contrib.layers.convolution2d(
    pool_1,
    num_outputs=32,
    weights_initializer=tf.truncated_normal_initializer(stddev=0.01),
    weights_regularizer = reg,
    kernel_size = (5,5),
    activation_fn = tf.nn.relu,
    stride = (1,1),
    padding='SAME',
    trainable = True)
pool_2 = tf.nn.max_pool(conv2d_2,
                       ksize = [1,3,3,1],
                       strides = [1,2,2,1],
                       padding='SAME')

In [9]:
conv_2d_w3 = tf.Variable(tf.truncated_normal([5, 5, 32, 64],stddev=0.01))
conv_2d_b3 =  tf.Variable(tf.truncated_normal([64]))
conv2d_3 = tf.nn.conv2d(pool_2, conv_2d_w3,strides=[1, 1, 1, 1], padding='SAME') + conv_2d_b3

In [10]:
conv2d_3_output =  tf.nn.relu(conv2d_3)

In [11]:
pool_3 = tf.nn.max_pool(conv2d_3_output,
                       ksize = [1,3,3,1],
                       strides = [1,2,2,1],
                       padding='SAME')

In [12]:
pool3_flat = tf.reshape(pool_3, [-1, 4*4*64])
fc_1 = tf.contrib.layers.fully_connected(
                        pool3_flat,
                        1024,
                        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                        weights_regularizer = reg,              
                        activation_fn = tf.nn.relu)

In [13]:
fc_2 = tf.contrib.layers.fully_connected(
                        fc_1,
                        128,
                        weights_initializer=tf.truncated_normal_initializer(stddev=0.1),
                        weights_regularizer = reg,              
                        activation_fn = tf.nn.relu)
fc2_drop = tf.nn.dropout(fc_2, keep_prob)

In [14]:
out_w1 = tf.Variable(tf.truncated_normal([128,10]))
out_b1 = tf.Variable(tf.truncated_normal([10]))
combine = tf.matmul(fc2_drop,out_w1)+out_b1
pred = tf.cast(tf.argmax(tf.nn.softmax(combine),1),tf.int32)

In [15]:
weights = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

In [16]:
reg_ws = tf.contrib.layers.apply_regularization(reg,weights_list = weights)

In [17]:
#reg_ws = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)

In [18]:
#update_avg = tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [19]:
loss = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_, logits=combine))

In [20]:
#loss = -tf.reduce_sum(y_*tf.log(tf.clip_by_value(pred,1e-11,1.0)))
loss_fn = loss + tf.reduce_sum(reg_ws)

In [21]:
#with tf.control_dependencies(update_avg):
train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss_fn)

In [22]:
accuracy = tf.reduce_mean(tf.cast(tf.equal(pred,y_), tf.float32))

In [23]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
data_list = []
label_list = []
for i in range(1,6):
    data = unpickle('./cifar-10/data_batch_{}'.format(i))
    data_list.append(data[b'data'])
    label_list.append(data[b'labels'])

In [ ]:
all_data = np.concatenate(data_list)
all_label = np.concatenate(label_list)

In [24]:
def generatebatch(X,Y,n_examples, batch_size):
    for batch_i in range(n_examples // batch_size):
        start = batch_i*batch_size
        end = start + batch_size
        batch_xs = X[start:end]
        batch_ys = Y[start:end]
        yield batch_xs, batch_ys # 生成每一个batch

In [27]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
ii = 0
for epoch in range(0,100): 
    index = np.random.permutation(all_label.shape[0])
    all_data = all_data[index]
    all_label = all_label[index]
    for batch_xs,batch_ys in generatebatch(all_data,all_label,all_label.shape[0],b_size): 
        batch_xs = np.array(list(map(lambda x:x.reshape([3,1024]).T.reshape([32,32,3]),batch_xs)))
        sess.run(train_step,feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 0.5})
        if ii%100 == 0:
            print(sess.run([loss,accuracy,],feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 1}))
        ii += 1
    if(epoch%2==0):
        res = sess.run([loss,accuracy],feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 1})
        print(epoch,res)
        saver.save(sess,'./lesson20',global_step = epoch)

[2230.3728, 0.1015625]
[296.13373, 0.1328125]
[299.21338, 0.15625]
[297.90588, 0.1796875]
0 [297.87201, 0.1640625]
[286.45709, 0.1484375]
[271.83194, 0.25]
[272.19818, 0.2578125]
[280.336, 0.28125]
[270.03656, 0.2265625]
[268.33881, 0.2421875]
[279.14911, 0.25]
[255.57489, 0.34375]
2 [269.97528, 0.3125]
[253.5652, 0.296875]
[270.59515, 0.1875]
[243.63385, 0.3671875]
[265.44107, 0.28125]
[277.95407, 0.296875]
[249.06958, 0.375]
[253.59618, 0.375]
[233.2317, 0.4375]
4 [236.92458, 0.34375]
[242.61093, 0.34375]
[239.72379, 0.40625]
[241.0367, 0.3203125]
[222.88277, 0.3828125]
[226.71214, 0.3671875]
[226.73242, 0.390625]
[244.57549, 0.359375]
[215.32364, 0.4921875]
6 [207.9684, 0.484375]
[225.48592, 0.453125]
[212.38525, 0.4765625]
[215.75592, 0.4375]
[218.75345, 0.3828125]
[211.10631, 0.53125]
[209.10144, 0.5078125]
[219.85703, 0.4609375]
[208.94769, 0.515625]
8 [214.26454, 0.34375]
[206.69048, 0.5234375]
[207.31659, 0.4453125]
[204.44267, 0.5]
[198.21962, 0.5078125]
[214.53455, 0.4609375]

KeyboardInterrupt: 

In [ ]:
saver.save(sess,'./lesson10',global_step=50000)

In [29]:
test = unpickle('./cifar-10/test_batch')

In [30]:
test_label_hot = test[b'labels']

In [31]:
test_data = test[b'data']

In [32]:
righ = []
for batch_xs,batch_ys in generatebatch(test_data,test_label_hot,test_data.shape[0],128): 
    batch_xs = np.array(list(map(lambda x:x.reshape([3,1024]).T.reshape([32,32,3]),batch_xs)))
    acc = sess.run(accuracy,feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 1})
    righ.append(acc)
print(sess.run(tf.reduce_mean(righ)))

0.788161


In [15]:
initial_step = 0

In [25]:
import os

In [26]:
ckpt = tf.train.get_checkpoint_state(os.path.dirname('__file__'))

In [27]:
ckpt.model_checkpoint_path = 'lesson20-90'

In [28]:
saver = tf.train.Saver()
sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver.restore(sess,ckpt.model_checkpoint_path)

INFO:tensorflow:Restoring parameters from lesson20-90


In [31]:
ii = 0
for epoch in range(49,400): 
    index = np.random.permutation(all_label.shape[0])
    all_data = all_data[index]
    all_label = all_label[index]
    for batch_xs,batch_ys in generatebatch(all_data,all_label,all_label.shape[0],b_size): 
        batch_xs = batch_xs.reshape((b_size,32,32,3))
        sess.run(train_step,feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 0.5})
        if ii%100 == 0:
            print(sess.run([loss,accuracy,],feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 1}))
        ii += 1
    if(epoch%2==0):
        res = sess.run([loss,accuracy],feed_dict={x: batch_xs, y_: batch_ys,keep_prob: 1})
        print(epoch,res)
        saver.save(sess,'./lesson9',global_step = epoch)

[115.66228, 0.58999997]
[104.24574, 0.64999998]
[104.00726, 0.55000001]
[124.28712, 0.51999998]
[100.06935, 0.64999998]
[100.69774, 0.62]
[81.071579, 0.70999998]
[103.37293, 0.67000002]
[103.27216, 0.67000002]
[90.464012, 0.69]
50 [109.93782, 0.61000001]
[103.62108, 0.67000002]
[102.42217, 0.63999999]
[111.91672, 0.56]
[93.016373, 0.69999999]
[102.25829, 0.64999998]
[114.70304, 0.61000001]
[103.18018, 0.62]
[94.557228, 0.67000002]
[112.92467, 0.66000003]
[108.07661, 0.64999998]
52 [101.67546, 0.64999998]
[104.90759, 0.66000003]
[106.28165, 0.62]
[115.44066, 0.58999997]
[102.48843, 0.64999998]
[93.553925, 0.69999999]
[104.64832, 0.68000001]
[91.337433, 0.69999999]
[102.34954, 0.70999998]
[94.392418, 0.73000002]
[116.42323, 0.56999999]
54 [93.844849, 0.67000002]
[103.3157, 0.61000001]
[111.61922, 0.62]
[105.12447, 0.67000002]
[112.5825, 0.63]
[124.54656, 0.55000001]
[110.49685, 0.63]
[106.44827, 0.66000003]
[92.969086, 0.64999998]
[74.30117, 0.79000002]
[121.38506, 0.60000002]
56 [100.77

KeyboardInterrupt: 

In [37]:
sess.run(tf.reduce_sum(reg_ws))

31.754757